In [1]:
import tensorflow as tf
import numpy as np
import datetime
import time

import sklearn.metrics

In [90]:
def genObs(length):
    return np.random.choice(range(5), size=length)

def genTarget(x):
    a = np.sum(x==1)
    b = np.sum(x==2)
    return (a, b, a * b)

def genSample(num, length=10):
    x = [genObs(length=length) for _ in range(num)]
    y = [genTarget(t) for t in x]
    return np.array(x), np.array(y)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

In [91]:
train_x, train_y = genSample(100000)
valid_x, valid_y = genSample(5000)
valid_x[:10], valid_y[:10]

(array([[2, 4, 4, 2, 0, 4, 3, 3, 3, 3],
        [1, 0, 0, 0, 1, 4, 0, 3, 1, 2],
        [4, 4, 3, 4, 2, 1, 2, 1, 0, 2],
        [1, 2, 4, 3, 2, 2, 2, 0, 4, 2],
        [2, 3, 2, 2, 4, 4, 3, 0, 4, 1],
        [4, 3, 1, 2, 2, 3, 4, 0, 0, 3],
        [4, 1, 1, 3, 3, 3, 0, 4, 1, 2],
        [4, 1, 4, 4, 0, 2, 1, 3, 1, 4],
        [3, 4, 0, 1, 4, 1, 4, 1, 2, 0],
        [4, 2, 0, 3, 1, 3, 2, 3, 0, 3]]), array([[0, 2, 0],
        [3, 1, 3],
        [2, 3, 6],
        [1, 5, 5],
        [1, 3, 3],
        [1, 2, 2],
        [3, 1, 3],
        [3, 1, 3],
        [3, 1, 3],
        [1, 2, 2]]))

In [92]:
SEQ_LEN = 10
NUM_TRGT = 3
RNN_SIZE = [10]

InnerCell = lambda n: tf.nn.rnn_cell.BasicRNNCell(num_units=n, activation=tf.nn.relu)
#InnerCell = lambda n: tf.nn.rnn_cell.LSTMCell(num_units=n, activation=tf.nn.relu, use_peepholes=True)

tf.reset_default_graph()

tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.float32)
tfi_y = tf.placeholder(shape=(None, NUM_TRGT), dtype=tf.int64)

tfX = tf.reshape(tfi_x, shape=(tf.shape(tfi_x)[0], tf.shape(tfi_x)[1], 1))
tfYC = tf.reshape(tfi_y, shape=(tf.shape(tfi_y)[0], NUM_TRGT))

rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE])
#rnnCell = tf.nn.rnn_cell.GRUCell(RNN_SIZE[0])

_, tfO = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32)

#tfFC0 = tf.concat([tfO[-1], tfO[-1][:,:5]/(1e-3 + tfO[-1][:,5:])], axis=1)
tfFC0 = tf.concat([tfO[-1], tfO[-1][:,:5] * (1.0+tfO[-1][:,5:])], axis=1)

tfOut = tf.layers.dense(tfFC0, 3)

tfLoss = tf.sqrt(tf.reduce_mean(tf.pow(tf.cast(tfYC[:,:], dtype=tf.float32) - tfOut[:,:], 2)))
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

tfOutR = tfOut[:,:]

#tfAccuracy = tf.reduce_mean(tf.cast(tf.equal(tfYC, tf.cast(tf.round(tfOut),dtype=tf.int64)), dtype=tf.float32))
tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [93]:
batch_size = 5000
num_steps  = 100
num_epochs = 50

fmt = 'Epoch {0} ({1:1.3} sec): loss changed from {2:1.3} to {3:1.3}\t\tVL:{4:1.3f}'

valid_batch = {tfi_x: valid_x, tfi_y: valid_y}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        mini_x, mini_y = randomBatch((train_x, train_y), batchSize=batch_size)
        train_batch = {tfi_x:mini_x, tfi_y:mini_y}
        l0 = tfLoss.eval(feed_dict=train_batch)
        t0 = time.perf_counter()
        for j in range(num_steps):
            tfTrain.run(feed_dict=train_batch)
        t1 = time.perf_counter()
        l1 = tfLoss.eval(feed_dict=train_batch)
        lv = tfLoss.eval(feed_dict=valid_batch)
        
        print(fmt.format(i,t1-t0,l0,l1,lv))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (1.45 sec): loss changed from 3.8 to 2.98		VL:3.012
Epoch 1 (1.45 sec): loss changed from 3.0 to 2.22		VL:2.232
Epoch 2 (1.45 sec): loss changed from 2.22 to 2.1		VL:2.124
Epoch 3 (1.45 sec): loss changed from 2.1 to 1.74		VL:1.760
Epoch 4 (1.45 sec): loss changed from 1.75 to 1.63		VL:1.633
Epoch 5 (1.46 sec): loss changed from 1.63 to 1.54		VL:1.551
Epoch 6 (1.6 sec): loss changed from 1.52 to 1.45		VL:1.481
Epoch 7 (1.54 sec): loss changed from 1.48 to 1.42		VL:1.427
Epoch 8 (1.45 sec): loss changed from 1.4 to 1.35		VL:1.385
Epoch 9 (1.45 sec): loss changed from 1.4 to 1.36		VL:1.350
Epoch 10 (1.45 sec): loss changed from 1.34 to 1.3		VL:1.323
Epoch 11 (1.45 sec): loss changed from 1.31 to 1.26		VL:1.289
Epoch 12 (1.46 sec): loss changed from 1.27 to 1.23		VL:1.261
Epoch 13 (1.49 sec): loss changed from 1.23 to 1.17		VL:1.226
Epoch 14 (1.48 sec): loss changed from 1.22 to 1.18		VL:1.187
Epoch 15 (1.45 sec): loss changed from 1.17 to 1.11		VL:1.141
Epoch 16 (1.45 sec): loss 

KeyboardInterrupt: 

In [84]:
valid_y[:10,2], valid_r[:10,2]

(array([ 3.        ,  0.66666667,  1.5       ,  0.4       ,  0.4       ,
         0.66666667,  0.5       ,  0.5       ,  1.5       ,  0.25      ]),
 array([ 2.18261909,  0.44013575,  1.27585876, -0.08560674,  0.20447388,
         0.36641315,  0.22130692,  0.24962249,  1.23564196,  0.20754772], dtype=float32))